In [1]:
import math

from mindspore import nn
from mindspore import ops
from mindspore.common.initializer import initializer

from mindnlp import load_dataset, process
from mindnlp.abc import Seq2vecModel
from mindnlp.engine import Trainer, Accuracy
from mindnlp.modules import Glove, RNNEncoder
from mindnlp.transforms import BasicTokenizer

[ERROR] ME(37345:140024222904512,MainProcess):2023-03-07-11:55:01.152.472 [mindspore/run_check/_check_version.py:226] Cuda ['10.1', '11.1', '11.6'] version(libcu*.so need by mindspore-gpu) is not found, please confirm that the path of cuda is set to the env LD_LIBRARY_PATH, or check whether the CUDA version in wheel package and the CUDA runtime in current device matches, please refer to the installation guidelines: https://www.mindspore.cn/install
[ERROR] ME(37345:140024222904512,MainProcess):2023-03-07-11:55:01.179.028 [mindspore/run_check/_check_version.py:226] Cuda ['10.1', '11.1', '11.6'] version(libcudnn*.so need by mindspore-gpu) is not found, please confirm that the path of cuda is set to the env LD_LIBRARY_PATH, or check whether the CUDA version in wheel package and the CUDA runtime in current device matches, please refer to the installation guidelines: https://www.mindspore.cn/install


In [2]:
# load embedding and vocab
embedding, vocab = Glove.from_pretrained('6B', 100, special_tokens=["<unk>", "<pad>"])

# load datasets
imdb_train, imdb_test = load_dataset('imdb', shuffle=True)
print(imdb_train.get_col_names())

['text', 'label']


In [3]:
tokenizer = BasicTokenizer(True)
imdb_train = process('imdb', imdb_train, tokenizer=tokenizer, vocab=vocab, \
                     bucket_boundaries=[400, 500], max_len=600, drop_remainder=True)
imdb_test = process('imdb', imdb_test, tokenizer=tokenizer, vocab=vocab, \
                     bucket_boundaries=[400, 500], max_len=600, drop_remainder=False)

In [4]:
class SentimentClassification(Seq2vecModel):
    def construct(self, text):
        _, (hidden, _), _ = self.encoder(text)
        context = ops.concat((hidden[-2, :, :], hidden[-1, :, :]), axis=1)
        output = self.head(context)
        return output

In [5]:
# Hyper-parameters
hidden_size = 256
output_size = 2
num_layers = 2
bidirectional = True
dropout = 0.5
lr = 5e-4

In [6]:
# build encoder
lstm_layer = nn.LSTM(100, hidden_size, num_layers=num_layers, batch_first=True,
                     dropout=dropout, bidirectional=bidirectional)
encoder = RNNEncoder(embedding, lstm_layer)

# build head
head = nn.SequentialCell([
    nn.Dropout(p=dropout),
    nn.Sigmoid(),
    nn.Dense(hidden_size * 2, output_size)
])

# build network
network = SentimentClassification(encoder, head)
loss = nn.CrossEntropyLoss()
optimizer = nn.Adam(network.trainable_params(), learning_rate=lr)

In [7]:
def initialize_weights(m):
    if isinstance(m, nn.Dense):
        m.weight.set_data(initializer('xavier_normal', m.weight.shape, m.weight.dtype))
        m.bias.set_data(initializer('zeros', m.bias.shape, m.bias.dtype))
    elif isinstance(m, nn.LSTM):
        for name, param in m.parameters_and_names():
            if 'bias' in name:
                param.set_data(initializer('zeros', param.shape, param.dtype))
            elif 'weight' in name:
                param.set_data(initializer('orthogonal', param.shape, param.dtype))

In [8]:
network.apply(initialize_weights)

SentimentClassification<
  (encoder): RNNEncoder<
    (embedding): Glove<
      (dropout_layer): Dropout<keep_prob=1.0>
      >
    (rnn): LSTM<
      (rnn): _DynamicLSTMCPUGPU<>
      (dropout_op): Dropout<p=0.5>
      >
    >
  (head): SequentialCell<
    (0): Dropout<p=0.5>
    (1): Sigmoid<>
    (2): Dense<input_channels=512, output_channels=2, has_bias=True>
    >
  >

In [9]:
# define metrics
metric = Accuracy()

# define trainer
trainer = Trainer(network=network, train_dataset=imdb_train, eval_dataset=imdb_test, metrics=metric,
                  epochs=5, loss_fn=loss, optimizer=optimizer)
trainer.run(tgt_columns="label")

Evaluate: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 393/393 [00:28<00:00, 13.82it/s]


Evaluate Score: {'Accuracy': 0.57068}


Evaluate: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 393/393 [00:29<00:00, 13.10it/s]


Evaluate Score: {'Accuracy': 0.71992}


Evaluate: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 393/393 [00:28<00:00, 13.57it/s]


Evaluate Score: {'Accuracy': 0.86732}


Evaluate: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 393/393 [00:29<00:00, 13.38it/s]


Evaluate Score: {'Accuracy': 0.87764}


Evaluate: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 393/393 [00:29<00:00, 13.40it/s]

Evaluate Score: {'Accuracy': 0.86912}
